# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [29]:
#Your code here
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [30]:
#Your code here
df = pd.read_csv ('homepage_actions.csv')
print(df)

                       timestamp      id       group action
0     2016-09-24 17:42:27.839496  804196  experiment   view
1     2016-09-24 19:19:03.542569  434745  experiment   view
2     2016-09-24 19:36:00.944135  507599  experiment   view
3     2016-09-24 19:59:02.646620  671993     control   view
4     2016-09-24 20:26:14.466886  536734  experiment   view
...                          ...     ...         ...    ...
8183  2017-01-18 09:11:41.984113  192060  experiment   view
8184  2017-01-18 09:42:12.844575  755912  experiment   view
8185  2017-01-18 10:01:09.026482  458115  experiment   view
8186  2017-01-18 10:08:51.588469  505451     control   view
8187  2017-01-18 10:24:08.629327  461199     control   view

[8188 rows x 4 columns]


In [31]:
df.tail()

,timestamp,id,group,action
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view
8187,2017-01-18 10:24:08.629327,461199,control,view


In [32]:
# df.info(), zero null values, id is an int, others are strings
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [33]:
df['action'].value_counts()
# 6328 'view', 1860 'click'

view     6328
click    1860
Name: action, dtype: int64

In [34]:
df['group'].value_counts()
# 4264 'control', 3924 'experiment'

control       4264
experiment    3924
Name: group, dtype: int64

In [35]:
df['id'].value_counts()

363314    2
368962    2
568156    2
267099    2
322257    2
         ..
419672    1
448358    1
761703    1
929641    1
196608    1
Name: id, Length: 6328, dtype: int64

In [36]:
df.groupby('id')['group'].value_counts()
# looks like some id's are accounted for twice in same group

id      group     
182988  experiment    1
182994  control       2
183089  control       1
183136  experiment    1
183141  experiment    2
                     ..
937003  control       1
937073  control       1
937108  control       1
937139  experiment    2
937217  control       2
Name: group, Length: 6328, dtype: int64

In [37]:
df.groupby('id')['action'].value_counts().head(10)
# looks like duplicate id's have both 'click' and 'view' while
# non duplicated id's are only 'view'

id      action
182988  view      1
182994  click     1
        view      1
183089  view      1
183136  view      1
183141  click     1
        view      1
183248  click     1
        view      1
183283  view      1
Name: action, dtype: int64

In [38]:
# so unique id's should be same as: all entries - clicks
len(df['id'].unique()) == (len(df)-len(df[df['action']=='click']))

True

In [39]:
df[df['id']==363314]

,timestamp,id,group,action
3751,2016-11-17 15:51:56.634476,363314,control,view
3752,2016-11-17 15:52:30.776850,363314,control,click


In [40]:
# breakdown of action by group
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [41]:
# breakdown of group by action
df.groupby('action')['group'].value_counts()

action  group     
click   control        932
        experiment     928
view    control       3332
        experiment    2996
Name: group, dtype: int64

In [42]:
# first lets state our hypotheses and Alpha level:

# Null hypothesis:
# The experimantal group did not have more clicks than the control group. 
# The experimental group had either the same or less clicks compared to the control group. 

# Alternative hypothesis:
# The experimental group's click rate is greater than the control group's, 
# thereby proving the experimantal homepage to be "more effective".

alpha = .05

In [43]:
# now, lets see how many were actually in each group
c_num = len(df[df['group']=='control']['id'].unique())
e_num = len(df[df['group']=='experiment']['id'].unique())

c_num, e_num
# (3332, 2996) still not equal in size

(3332, 2996)

In [44]:
# how many clicks (successes) from each group
c_success = len(df[(df['group']=='control')&(df['action']=='click')])
e_success = len(df[(df['group']=='experiment')&(df['action']=='click')])

c_success, e_success
# (932, 928)

(932, 928)

In [45]:
# percent of clicks in each group 
c_clicks_percent = c_success/c_num
e_clicks_percent = e_success/e_num

round(c_clicks_percent, 2), round(e_clicks_percent, 2)
# 3% more from the experimantal page, but is that significant?

(0.28, 0.31)

In [54]:
# maybe eaier way would be to drop duplicated id's
# we know click comes after view so we can keep last
df = df.drop_duplicates(subset=['id'], keep='last',\
                        inplace=False, ignore_index=False)
df.tail(10)

,timestamp,id,group,action
8176,2017-01-18 07:07:50.090346,540466,experiment,view
8177,2017-01-18 07:15:05.620801,839581,control,view
8178,2017-01-18 08:17:12.675797,616692,control,view
8180,2017-01-18 08:54:56.879682,615849,experiment,click
8182,2017-01-18 09:09:17.363917,795585,control,click
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view
8187,2017-01-18 10:24:08.629327,461199,control,view


In [55]:
# drop timestamp column, its not needed, at this point we could
# techniqually drop id too however I chose to keep it
df = df.drop(columns='timestamp')
df.head()

,id,group,action
0,804196,experiment,view
1,434745,experiment,view
2,507599,experiment,view
3,671993,control,view
4,536734,experiment,view


In [56]:
# replace action values to 0 & 1
df = df.replace({'view': 0, 'click': 1})
df.tail(10)

,id,group,action
8176,540466,experiment,0
8177,839581,control,0
8178,616692,control,0
8180,615849,experiment,1
8182,795585,control,1
8183,192060,experiment,0
8184,755912,experiment,0
8185,458115,experiment,0
8186,505451,control,0
8187,461199,control,0


In [57]:
# seperate into two df's
c = df[df['group']=='control']
e = df[df['group']=='experiment']

In [58]:
# check if the values acquired two different ways are equal
c['action'].mean() == c_clicks_percent

# True, they are equal
# now that I have the values in dfs instead of numbers (len())
# I can use stats.ttest_ind to find p-value

True

In [59]:
# find p-value
t_stat, p_value = stats.ttest_ind(c['action'], e['action'], equal_var=False, nan_policy='omit') # equal_var=False: Welch's T-test
oneside_pval = p_value/2
oneside_pval
#0.004466402814337102

0.004466402814337102

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [46]:
#Your code here
# percent of control clicks
c_clicks_percent

# if the percent of control matches percent of experiment then 
expected_e_clicks = c_clicks_percent*e_num

# experiment would have ---- of clicks
expected_e_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [50]:
#Your code here
# var = n * p * (1-p)
# n = number in experiment group
# p = probability (in this case, expected p)
var = e_num * c_clicks_percent * (1-c_clicks_percent)
std = np.sqrt(var)
std

24.568547907005815

In [51]:
actual_e_clicks = e_success

# z-score
z_score = (actual_e_clicks - expected_e_clicks)/std

z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [52]:
#Your code here
p_val = stats.norm.sf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [60]:
# compare two p-values acquired two different ways
oneside_pval, p_val

(0.004466402814337102, 0.00012486528006951198)

In [61]:
p_val < alpha

True

In [62]:
oneside_pval < alpha

True

In [63]:
# while they are not exactly the same they are both well below the
# alpha, meaning the null hypothesis should be rejected. 

# The experimental group's click rate is greater than the control 
# group's, thereby proving the experimental homepage to be "more 
# effective".

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.